In [ ]:
그래프 모양/설정을 바꾸고 싶으면 -> plot_func.py
데이터 읽는 방식/점수 계산/트리 구조를 바꾸고 싶으면 -> data_load.py
화면 디자인을 바꾸고 싶으면 -> static/css/style.css
화면 클릭 동작/D3 로직을 바꾸고 싶으면 -> static/js/script.js

project_root/
├── storage/             # (기존 데이터 폴더)
├── static/              # 정적 파일 (CSS, JS)
│   ├── css/
│   │   └── style.css
│   └── js/
│       └── script.js
├── templates/
│   └── index.html
├── data_load.py         # 데이터 로드, 병합, 트리 생성 로직
├── plot_func.py         # 차트 설정(Plotly) 관련 함수
├── main.py              # FastAPI 서버 엔드포인트
└── init_storage.py      # (데이터 생성용 스크립트 - 그대로 둠)

In [ ]:
실행 테스트

1. process 변경시
index.html(메뉴판)의 onchange -> script.js의 updateToolList(웨이터) -> main.py의 get_tools_api(지배인) -> data_load.py의 get_tool_list(요리사) 

In [2]:
# init_storage.py
import os
import pandas as pd
import random
import datetime
import shutil

# 기존 storage 폴더가 있다면 삭제 후 재생성 (초기화)
if os.path.exists("storage"):
    shutil.rmtree("storage")

BASE_DIR = "storage"
PROCESSES = ["process1", "process2", "process3"]
TOOLS = ["toolA", "toolB", "toolC"] # 각 공정마다 들어갈 툴들
RECIPES = ["Recipe_X", "Recipe_Y", "Recipe_Z"]

# 센서 정의
sensors = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
groups = ['temp', 'temp', 'gas', 'gas', 'press', 'press']

start_date = datetime.date(2025, 12, 1)
end_date = datetime.date(2025, 12, 10)

print("데이터 생성 시작...")

for proc in PROCESSES:
    for tool in TOOLS:
        # 폴더 경로: ./storage/process1/toolA
        dir_path = os.path.join(BASE_DIR, proc, tool)
        os.makedirs(dir_path, exist_ok=True)
        
        current = start_date
        while current <= end_date:
            date_str = current.strftime("%Y-%m-%d")
            
            # 날짜별로 랜덤한 레시피 생성 (안 만드는 날도 있음)
            if random.random() > 0.2: 
                daily_recipes = random.sample(RECIPES, k=random.randint(1, 2))
                
                for recipe in daily_recipes:
                    # 파일명: 툴_날짜_레시피.parquet
                    # 예: toolA_2025-12-01_Recipe_X.parquet
                    filename = f"{tool}_{date_str}_{recipe}.parquet"
                    filepath = os.path.join(dir_path, filename)
                    
                    rows = []
                    for s, g in zip(sensors, groups):
                        base = random.randint(30, 80)
                        history = [base + random.randint(-5, 5) for _ in range(20)]
                        rows.append({
                            "sensor": s,
                            "group": g,
                            "history": history,
                            "value": history[-1]
                        })
                    
                    df = pd.DataFrame(rows)
                    df.to_parquet(filepath, engine='pyarrow')
                    print(f"Created: {filepath}")
            
            current += datetime.timedelta(days=1)

print("\n초기 데이터 생성 완료! main.py를 실행하세요.")

데이터 생성 시작...
Created: storage\process1\toolA\toolA_2025-12-01_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-02_Recipe_X.parquet
Created: storage\process1\toolA\toolA_2025-12-03_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-03_Recipe_X.parquet
Created: storage\process1\toolA\toolA_2025-12-04_Recipe_X.parquet
Created: storage\process1\toolA\toolA_2025-12-04_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-05_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-05_Recipe_Y.parquet
Created: storage\process1\toolA\toolA_2025-12-06_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-06_Recipe_Y.parquet
Created: storage\process1\toolA\toolA_2025-12-07_Recipe_Y.parquet
Created: storage\process1\toolA\toolA_2025-12-07_Recipe_Z.parquet
Created: storage\process1\toolA\toolA_2025-12-08_Recipe_Y.parquet
Created: storage\process1\toolA\toolA_2025-12-10_Recipe_Y.parquet
Created: storage\process1\toolB\toolB_2025-12-01_Recipe_Z.parqu

In [3]:
# init_storage.py
import os
import pandas as pd
import random
import datetime
import shutil

# 초기화
if os.path.exists("storage"):
    shutil.rmtree("storage")

BASE_DIR = "storage"
SCORE_DIR = os.path.join(BASE_DIR, "score")
PROCESSES = ["process1", "process2", "process3"]
TOOLS = ["toolA", "toolB", "toolC"] 
RECIPES = ["Recipe_X", "Recipe_Y", "Recipe_Z"]

# 센서 정의
sensors = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
groups = ['temp', 'temp', 'gas', 'gas', 'press', 'press']

start_date = datetime.date(2025, 12, 1)
end_date = datetime.date(2025, 12, 10)

print("데이터 생성 시작...")

# 1. 일반 센서 데이터 생성 (기존 로직)
for proc in PROCESSES:
    for tool in TOOLS:
        dir_path = os.path.join(BASE_DIR, proc, tool)
        os.makedirs(dir_path, exist_ok=True)
        
        current = start_date
        while current <= end_date:
            date_str = current.strftime("%Y-%m-%d")
            if random.random() > 0.2: 
                daily_recipes = random.sample(RECIPES, k=random.randint(1, 2))
                for recipe in daily_recipes:
                    filename = f"{tool}_{date_str}_{recipe}.parquet"
                    filepath = os.path.join(dir_path, filename)
                    
                    rows = []
                    for s, g in zip(sensors, groups):
                        base = random.randint(30, 80)
                        history = [base + random.randint(-5, 5) for _ in range(20)]
                        rows.append({
                            "sensor": s,
                            "group": g,
                            "history": history,
                            "value": history[-1]
                        })
                    pd.DataFrame(rows).to_parquet(filepath, engine='pyarrow')
            current += datetime.timedelta(days=1)

# 2. Score 데이터 생성 (신규 로직)
os.makedirs(SCORE_DIR, exist_ok=True)
print("Score 데이터 생성 중...")

current = start_date
while current <= end_date:
    date_str = current.strftime("%Y-%m-%d")
    
    # 해당 날짜의 툴별 점수 데이터 생성
    # 컬럼: ToolName, time, top temp, bottom temp ...
    score_rows = []
    for tool in TOOLS:
        row = {
            "ToolName": tool,
            "time": date_str
        }
        # 센서별 점수 난수 (50 ~ 100점)
        for s in sensors:
            row[s] = random.randint(50, 100)
        score_rows.append(row)
    
    score_df = pd.DataFrame(score_rows)
    
    # 파일명: YYYY-MM-DD_score.parquet
    score_filepath = os.path.join(SCORE_DIR, f"{date_str}_score.parquet")
    score_df.to_parquet(score_filepath, engine='pyarrow')
    print(f"Score Created: {score_filepath}")
    
    current += datetime.timedelta(days=1)

print("\n데이터 생성 완료! main.py를 실행하세요.")

데이터 생성 시작...
Score 데이터 생성 중...
Score Created: storage\score\2025-12-01_score.parquet
Score Created: storage\score\2025-12-02_score.parquet
Score Created: storage\score\2025-12-03_score.parquet
Score Created: storage\score\2025-12-04_score.parquet
Score Created: storage\score\2025-12-05_score.parquet
Score Created: storage\score\2025-12-06_score.parquet
Score Created: storage\score\2025-12-07_score.parquet
Score Created: storage\score\2025-12-08_score.parquet
Score Created: storage\score\2025-12-09_score.parquet
Score Created: storage\score\2025-12-10_score.parquet

데이터 생성 완료! main.py를 실행하세요.
